# RNNs

En este laboratorio aprenderás a usar distintos tipos de redes recurrentes y la
aplicarás para clasificar un conjunto de reviews de películas de IMDB en
"positivo" o "negativo". Partimos importando los datos


In [1]:
import numpy as np
from tensorflow import keras
import tensorflow as tf

Cargamos los datos


In [2]:
max_palabras = 30000
a_train = 15000


(X_train, Y_train), (X_test, Y_test) = tf.keras.datasets.imdb.load_data(
    num_words=max_palabras
)

# modificamos un poco los datasets para tener 40000 de train y 10000 de test
X_train = np.concatenate((X_train, X_test[:a_train]), axis=0)
Y_train = np.concatenate((Y_train, Y_test[:a_train]), axis=0)

X_test = X_test[a_train:]
Y_test = Y_test[a_train:]

print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
(40000,) (40000,)
(10000,) (10000,)


Estas son funciones auxiliares para manejar textos y transformarlos en índices
para poder trabajar más fácilmente con ellos. Si sabes un poco de python y
tienes tiempo, dales una mirada. Son cosas standard que se deben hacer para
procesar texto antes y después de pasárselos a una red neuronal.


In [36]:
palabra_a_id = keras.datasets.imdb.get_word_index()
palabra_a_id = {k: (v + 3) for k, v in palabra_a_id.items()}
id_a_palabra = {v: k for k, v in palabra_a_id.items()}
id_a_palabra[0] = "<PAD>"
id_a_palabra[1] = "<INI>"
id_a_palabra[2] = "<NN>"


def ids_a_texto(lista):
    lista_palabras = [id_a_palabra[id] for id in lista]
    texto = " ".join(lista_palabras)
    return texto


def texto_a_ids(texto):
    lista_palabras = texto.split()
    lista = [1] + [palabra_a_id[w] if w in palabra_a_id else 2 for w in lista_palabras]
    return lista


def texto_a_red_input(texto, max_largo=80):
    lista = texto_a_ids(texto)
    X = np.array(lista).reshape(1, -1)
    X = keras.preprocessing.sequence.pad_sequences(X, maxlen=max_largo)
    return X


def posneg(i):
    if i < 0.5:
        return "negativo"
    else:
        return "positivo"

Hacemos un preprocesamiento de los datos para que todos los textos tengan el
mismo largo (80 en este caso).


In [ ]:
max_largo = 80

# mantenemos los originales
X_train_original = X_train
X_test_original = X_test

# ahora los procesamos
X_train = keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_largo)
X_test = keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_largo)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

Ejemplos de textos y sus etiquetas


In [ ]:
N = 5
for _ in range(N):
    i = np.random.randint(len(X_test))
    print("original:", ids_a_texto(X_test_original[i]))
    print("input red:", ids_a_texto(X_test[i]))
    print("etiqueta:", posneg(Y_test[i]))
    print()

Importamos las capas recurrentes que usaremos


In [40]:
from keras.layers import SimpleRNN, GRU, LSTM, Embedding, Dense, Dropout, Bidirectional

Creamos nuestra primera red recurrente.


In [7]:
np.random.seed(30)
tf.random.set_seed(30)

emb_dim = 32

rnn = keras.Sequential()
rnn.add(Embedding(max_palabras, emb_dim, input_length=max_largo))
rnn.add(SimpleRNN(32))
rnn.add(Dense(1, activation="sigmoid"))

# rnn.summary()

Compilamos y entrenamos nuestra red (solo por una época, si quieres puedes usar
más pero las siguientes tardarán un tiempo considerablemente mayor). Usamos Adam
como optimizador para mejorar las posibilidades de obtener una buena solución
con pocas iteraciones (solo en una época).


In [ ]:
rnn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
rnn.fit(X_train, Y_train, batch_size=32, epochs=1, validation_data=(X_test, Y_test))

Nuestro primer modelo llega a una certeza cercana al 83%.

Mostramos las predicciones para algunos ejemplos. Note que ahora podemos pasarle
como ejemplos textos propios para que pueda predecir. Si bien la certeza en el
conjunto de prueba es bastante alta, las predicciones para ejemplos "difíciles"
no son muy buenas.


In [9]:
textos = [
    "this is a wonderful movie",
    "this is a totally awful movie",
    "this is an awful movie i really do not recommend it",
    "this is the worst movie ever",
    "i was expecting more way more",
    "not what i expected but at the end i enjoyed it",
    "i will see it again",
    "i won't see it again",
    "this is an excellent movie",
    "if you are an idiot this is an excellent movie",
]

In [ ]:
for texto in textos:
    X_in = texto_a_red_input(texto, max_largo)
    pred = rnn.predict(X_in)
    print(texto)
    print(posneg(pred.item()), pred.item())
    print()

# Ejercicio RNNs

Trata de llegar a un 86% de certeza en el conjunto de prueba. Para esto prueba
distintas opciones de arquitecturas (ojalá pruebes al menos 3 de creciente
complejidad, sin olvidar la regularización). Comienza aumentando las neuronas
del embedding a 64 y luego prueba algunas de las siguientes opciones:

- RNN con más neuronas (en embedding o en la capa recurrente)
- GRU o LSTM
- GRU o LSTM bidireccional
- dos (o mas?) capas de GRUs o LSTMs

Si tienes más tiempo también puedes probar cambiando otros hiperparámetros
(aunque sería bueno que para las primeras redes te fijes más que nada en la
arquitectura).

y similar para `GRU` y `LSTM`. Para poder poner una capa recurrente sobre otra
debes usar el parámetro `return_sequences=True`. Con esto le dices a la red que
esperas que retorne la capa escondida en cada iteración y no solo al final. Por
ejemplo el siguiente trozo de código usa tres capas recurrentes una sobre otra

```
...
red.add(SimpleRNN(32, return_sequences=True))
red.add(SimpleRNN(16, return_sequences=True))
red.add(SimpleRNN(8))
red.add(Dense(1))
...
```

la primera de recurrencia simple y 32 neuronas, la segunda es una capa de 16
neuronas y la ultima una capa simple de 8 neuronas. Nota que la última capa
recurrente no necesita el parámetro `return_sequences=True` pues solo se usará
la capa escondida final para pasarla por la última capa densa.

Para cada red que crees, calcula el acierto en el conjunto de prueba y muestra
algunas predicciones de frases escritas por ti para ver cómo se comporta en
distintos casos. De los casos complicados de arriba, las predicciones deberían
empezar a mejorar (al menos respecto de la probabilidad que le asigna a ser
positivo o negativo).


In [ ]:
# Acá comienza todo tu código